# Predicting Poisonous Mushrooms

**Highlights:**
> 1. Correlation analysis using Cramer's V
>
> 2. Results for each ML algorithm are presented after performing 5-fold cross validation based on F1-score

* [1. Data Munging](#data-munging)
    * [1.1. Checking for null values in the dataset](#checking-for-null-values-in-the-dataset)
    * [1.2. Search for categorical columns and cast them to `pd.Categorical`](#search-for-categorical-columns-and-cast-them-to-%60pd.categorical%60)
    * [1.3. Delete `veil-type` column](#delete-%60veil-type%60-column)
    * [1.4. Reordering Columns](#reordering-columns)
* [2. Correlations in the data](#correlations-in-the-data)
* [3. Data Visualization](#data-visualization)
    * [3.1. Frequency distribution](#frequency-distribution)
    * [3.2. Distribution among different habitat](#distribution-among-different-habitat)
    * [3.3. Distribution among different populations types](#distribution-among-different-populations-types)
    * [3.4. Distribution among different habitat and population types taken together](#distribution-among-different-habitat-and-population-types-taken-together)
* [4. Data Preprocessing](#data-preprocessing)
    * [4.1. Train-Test split](#train-test-split)
    * [4.2. One-hot Encoding](#one-hot-encoding)
* [5. Data Modeling](#data-modeling)
    * [5.1. Utility Functions](#utility-functions)
    * [5.2. Naive Bayes](#naive-bayes)
    * [5.3. Logistic Regression](#logistic-regression)
    * [5.4. K-Nearest Neighbors](#k-nearest-neighbors)
    * [5.5. Decision Tree](#decision-tree)
    * [5.6. Decision Trees with Bagging](#decision-trees-with-bagging)
    * [5.7. Random Forests](#random-forests)
    * [5.8. Decision Trees with AdaBoost](#decision-trees-with-adaboost)
    * [5.9. Linear SVC](#linear-svc)
    * [5.10. SVM with RBF kernel](#svm-with-rbf-kernel)
    * [5.11. XGBoost](#xgboost)
    * [5.12. CatBoost](#catboost)
* [6. Model Comparison](#model-comparison)
    * [6.1. Evaluation Metrics](#evaluation-metrics)
    * [6.2. ROC and PR Curves](#roc-and-pr-curves)

In [ ]:
import os, sys, json

import numpy as np
from scipy.stats import chi2_contingency
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# Get full labels from JSON file
with open('/kaggle/input/mushroom-labels/full_labels.json', 'r') as fp:
    full_labels = json.load(fp)

In [ ]:
df = pd.read_csv('/kaggle/input/mushroom-classification/mushrooms.csv')

In [ ]:
df.head(2)

In [ ]:
df.info()

In [ ]:
df = df.apply(func=lambda ser: ser.replace(to_replace=full_labels[ser.name]),
              axis=0)

<a id="data-munging"></a>
# 1. Data Munging

<a id="checking-for-null-values-in-the-dataset"></a>
## 1.1. Checking for null values in the dataset
There are no null values in the dataset.

In [ ]:
df[df.isnull().any(axis=1)]

<a id="search-for-categorical-columns-and-cast-them-to-%60pd.categorical%60"></a>
## 1.2. Search for categorical columns and cast them to `pd.Categorical`
We need to manually identify categorical columns in the data before casting them to `pd.Categorical`. Casting categorical columns from the detected *object* type to *categorical* will ease visualization.

In [ ]:
def summarize_categoricals(df, show_levels=False):
    """ Display uniqueness in each column """
    df_temp = pd.DataFrame([[df[col].unique(), len(df[col].unique()), df[col].isnull().sum()] for col in df.columns],
                           index=df.columns, columns=['Levels', 
                                                      'No. of Levels',
                                                      'No. of Missing Values'])
    return df_temp.iloc[:, 0 if show_levels else 1:]


def find_categorical(df, cutoff=10):
    """ Function to find categorical columns in the dataframe. """
    cat_cols = []
    for col in df.columns:
        if len(df[col].unique()) <= cutoff:
            cat_cols.append(col)
    return cat_cols


def to_categorical(columns, df):
    """ Converts the columns passed in `columns` to categorical datatype. """
    for col in columns:
        df[col] = df[col].astype('category')
    return df

In [ ]:
summarize_categoricals(df, show_levels=True)

In [ ]:
df = to_categorical(find_categorical(df, cutoff=12), df)

df.info()

<a id="delete-%60veil-type%60-column"></a>
## 1.3. Delete `veil-type` column
Since `veil-type` feature does not have any variance, it does not have any discriminative power. In otherwords, since `veil-type` feature has the same value for all the samples (see section 1.2 categorical summary table), it holds no information that facilitates differentiating/ separating the target classes. Therefore, we can delete this features without losing any information.

In [ ]:
df.drop(labels=['veil-type'], axis=1, inplace=True)

df.info()

<a id="reordering-columns"></a>
## 1.4. Reordering Columns

In [ ]:
# Moving the target column to the end
new_order = list(df.columns)
new_order.append(new_order.pop(0))
df = df[new_order]
df.info()

<a id="correlations-in-the-data"></a>
# 2. Correlations in the data

In [ ]:
df.describe().T

`Cramer's V` is more appropriate than Pearson correlation to find correlation between two nominal variables. Here, the `Cramer's V` metric is implemented.

In [ ]:
def cramers_corrected_stat(contingency_table):
    """
        Computes corrected Cramer's V statistic for categorial-categorial association
        Taken from here:
        https://stackoverflow.com/questions/20892799/using-pandas-calculate-cram%C3%A9rs-coefficient-matrix
    """
    chi2 = chi2_contingency(contingency_table)[0]
    n = contingency_table.sum().sum()
    phi2 = chi2/n
    
    r, k = contingency_table.shape
    r_corrected = r - (((r-1)**2)/(n-1))
    k_corrected = k - (((k-1)**2)/(n-1))
    phi2_corrected = max(0, phi2 - ((k-1)*(r-1))/(n-1))
    
    return (phi2_corrected / min( (k_corrected-1), (r_corrected-1)))**0.5

In [ ]:
def categorical_corr_matrix(df):
    """
        Computes corrected Cramer's V statistic between all the categorical variables in the dataframe
    """
    df = df.select_dtypes(include='category')
    cols = df.columns
    n = len(cols)
    corr_matrix = pd.DataFrame(np.zeros(shape=(n, n)), index=cols, columns=cols)
    
    for col1 in cols:
        for col2 in cols:
            if col1 == col2:
                corr_matrix.loc[col1, col2] = 1
                break
            df_crosstab = pd.crosstab(df[col1], df[col2], dropna=False)
            corr_matrix.loc[col1, col2] = cramers_corrected_stat(df_crosstab)
    
    # Flip and add to get full correlation matrix
    corr_matrix += np.tril(corr_matrix, k=-1).T
    return corr_matrix

In [ ]:
fig, ax = plt.subplots(figsize=(15, 10))
sns.heatmap(categorical_corr_matrix(df), annot=True, cmap='coolwarm', 
            cbar_kws={'aspect': 50}, square=True, ax=ax)
plt.xticks(rotation=30, ha='right');
plt.tight_layout()

***Inference:*** There is a very high correlation (0.97) between the `color` of a mushroom and whether it is poisonous or not. This means we can correctly predict the class for almost 97% of the samples using just the `color` feature.

<a id="data-visualization"></a>
# 3. Data Visualization

<a id="frequency-distribution"></a>
## 3.1. Frequency distribution

In [ ]:
fig, axs = plt.subplots(nrows=8, ncols=3, figsize=(15, 20))
ax_title_pairs = zip(axs.flat, list(df.columns))

for ax, title in ax_title_pairs:
    sns.countplot(x=title, data=df, palette='Pastel2', ax=ax)
    ax.set_title(title.title())
    ax.set_xticklabels(ax.get_xticklabels(), rotation=30)
    ax.set_xlabel('')

axs[7][1].set_axis_off()
axs[7][2].set_axis_off()
plt.tight_layout()

***Inference:*** The dataset is almost balanced.

<a id="distribution-among-different-habitat"></a>
## 3.2. Distribution among different habitat

In [ ]:
display(pd.crosstab(df['class'], df['habitat'], dropna=False))
sns.countplot(x='class', hue='habitat', data=df, palette='Pastel2')

# Put the legend out of the figure
plt.legend(title='Habitat', bbox_to_anchor=(1, 1))

<a id="distribution-among-different-populations-types"></a>
## 3.3. Distribution among different populations types

In [ ]:
display(pd.crosstab(df['class'], df['population'], dropna=False))
sns.countplot(x='class', hue='population', data=df, palette='Pastel2')

# Put the legend out of the figure
plt.legend(title='population', bbox_to_anchor=(1, 1))

<a id="distribution-among-different-habitat-and-population-types-taken-together"></a>
## 3.4. Distribution among different habitat and population types taken together
Modifying seaborn countplot make it work with FacetGrid when all 3 arguments (`hue`, `row`, and `col`) are used.

In [ ]:
def modified_countplot(**kargs):
    """
        Assumes that columns to be plotted are in of pandas dtype='CategoricalDtype'
    """
    facet_gen = kargs['facet_generator']    ## Facet generator over facet data
    curr_facet, facet_data = None, None
    
    while True:
        ## Keep yielding until non-empty dataframe is found
        curr_facet = next(facet_gen)            ## Yielding facet genenrator
        df_rows = curr_facet[1].shape[0]
        
        ## Skip the current facet if its corresponding dataframe empty
        if df_rows:
            facet_data = curr_facet[1]
            break
    
    x_hue = (kargs.get('x'), kargs.get('hue'))
    cols = [col for col in x_hue if col]
    col_categories = [facet_data[col].dtype.categories if col else None for col in x_hue]
    
    palette = kargs['palette'] if 'palette' in kargs.keys() else 'Pastel2'
    sns.countplot(x=cols[0], hue=x_hue[1], 
                  order=col_categories[0], hue_order=col_categories[1],
                  data=facet_data.loc[:, cols], palette=palette)

In [ ]:
facet = sns.FacetGrid(df, row='habitat', col='class', 
                      sharex=False, sharey=False, margin_titles=True)
facet.map(modified_countplot, x='population',
          palette='Pastel2', facet_generator=facet.facet_data())
facet.set_xlabels('Population')
facet.set_ylabels('Count')
facet.set_xticklabels(df['population'].dtype.categories, rotation=30)
plt.tight_layout()

<a id="data-preprocessing"></a>
# 4. Data Preprocessing
Data needs to be one-hot-encoded before applying machine learning models.

In [ ]:
x = df.iloc[:, :-1]
y = df['class']

feature_names = list(x.select_dtypes(include='category').columns)

<a id="train-test-split"></a>
## 4.1. Train-Test split
CatBoost classifier does not require any knd of preprocessing while Naive bayes requires a different kind of preprocesing. Therefore, we will use raw/ unmodified data (`x_train_cat, x_test_cat, y_train_cat, y_test_cat`) for CatBoost and preprocessed data (`x_train, x_test, y_train, y_test`) for all other classifiers. For Naive Bayes, we will use the raw data (`x_train_cat, x_test_cat, y_train_cat, y_test_cat`) and preprocess it as required in the Naive Bayes section.

In [ ]:
from sklearn.model_selection import train_test_split

data_splits = train_test_split(x, y, test_size=0.25, random_state=0,
                               shuffle=True, stratify=y)
x_train, x_test, y_train, y_test = data_splits

# For CatBoost and Naive Bayes
data_splits = train_test_split(x, y, test_size=0.25, random_state=0,
                               shuffle=True, stratify=y)
x_train_cat, x_test_cat, y_train_cat, y_test_cat = data_splits

list(map(lambda x: x.shape, [x, y, x_train, x_test, y_train, y_test]))

In [ ]:
pd.Series(y_test).value_counts()

In [ ]:
sns.countplot(x=y_test)

<a id="one-hot-encoding"></a>
## 4.2. One-hot Encoding

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, LabelEncoder

## Column Transformer
x_trans = ColumnTransformer([('one_hot_encoder',
                              OneHotEncoder(drop='first',dtype='int'),
                              feature_names)],
                            remainder='passthrough')

x_train = x_trans.fit_transform(x_train)
x_test = x_trans.transform(x_test)

## Save feature names after one-hot encoding for feature importances plots
feature_names = list(x_trans.named_transformers_['one_hot_encoder'] \
                            .get_feature_names(input_features=feature_names))

## Label encoder
y_trans = LabelEncoder()
y_train = y_trans.fit_transform(y_train)
y_test = y_trans.transform(y_test)

y_trans.transform(['poisonous', 'edible'])

<a id="data-modeling"></a>
# 5. Data Modeling
Since the dataset is imbalanced we will be using class-weighted/ cost-sensitive learning. In cost-sensitive learning, a weighted cost function is used. Therefore, misclassifying a sample from the minority class will cost the classifiers more than misclassifying a sample from the majority class. In most of the Sklearn classifiers, cost-sensitive learning can be enabled by setting `class_weight='balanced'`.

<a id="utility-functions"></a>
## 5.1. Utility Functions

In [ ]:
import timeit
import pickle
import sys
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score, \
                            precision_recall_curve, roc_curve, accuracy_score
from sklearn.exceptions import NotFittedError

In [ ]:
def confusion_plot(matrix, labels=None):
    """ Display binary confusion matrix as a Seaborn heatmap """
    
    labels = labels if labels else ['Negative (0)', 'Positive (1)']
    
    fig, ax = plt.subplots(nrows=1, ncols=1)
    sns.heatmap(data=matrix, cmap='Blues', annot=True, fmt='d',
                xticklabels=labels, yticklabels=labels, ax=ax)
    ax.set_xlabel('PREDICTED')
    ax.set_ylabel('ACTUAL')
    ax.set_title('Confusion Matrix')
    plt.close()
    
    return fig

In [ ]:
def roc_plot(y_true, y_probs, label, compare=False, ax=None):
    """ Plot Receiver Operating Characteristic (ROC) curve 
        Set `compare=True` to use this function to compare classifiers. """
    
    fpr, tpr, thresh = roc_curve(y_true, y_probs, drop_intermediate=False)
    auc = round(roc_auc_score(y_true, y_probs), 2)
    
    fig, axis = (None, ax) if ax else plt.subplots(nrows=1, ncols=1)
    label = ' '.join([label, f'({auc})']) if compare else None
    sns.lineplot(x=fpr, y=tpr, ax=axis, label=label)
    
    if compare:
        axis.legend(title='Classifier (AUC)', loc='lower right')
    else:
        axis.text(0.72, 0.05, f'AUC = { auc }', fontsize=12,
                  bbox=dict(facecolor='green', alpha=0.4, pad=5))
            
        # Plot No-Info classifier
        axis.fill_between(fpr, fpr, tpr, alpha=0.3, edgecolor='g',
                          linestyle='--', linewidth=2)
        
    axis.set_xlim(0, 1)
    axis.set_ylim(0, 1)
    axis.set_title('ROC Curve')
    axis.set_xlabel('False Positive Rate [FPR]\n(1 - Specificity)')
    axis.set_ylabel('True Positive Rate [TPR]\n(Sensitivity or Recall)')
    
    plt.close()
    
    return axis if ax else fig

In [ ]:
def precision_recall_plot(y_true, y_probs, label, compare=False, ax=None):
    """ Plot Precision-Recall curve.
        Set `compare=True` to use this function to compare classifiers. """
    
    p, r, thresh = precision_recall_curve(y_true, y_probs)
    p, r, thresh = list(p), list(r), list(thresh)
    p.pop()
    r.pop()
    
    fig, axis = (None, ax) if ax else plt.subplots(nrows=1, ncols=1)
    
    if compare:
        sns.lineplot(r, p, ax=axis, label=label)
        axis.set_xlabel('Recall')
        axis.set_ylabel('Precision')
        axis.legend(loc='lower left')
    else:
        sns.lineplot(thresh, p, label='Precision', ax=axis)
        axis.set_xlabel('Threshold')
        axis.set_ylabel('Precision')
        axis.legend(loc='lower left')

        axis_twin = axis.twinx()
        sns.lineplot(thresh, r, color='limegreen', label='Recall', ax=axis_twin)
        axis_twin.set_ylabel('Recall')
        axis_twin.set_ylim(0, 1)
        axis_twin.legend(bbox_to_anchor=(0.24, 0.18))
    
    axis.set_xlim(0, 1)
    axis.set_ylim(0, 1)
    axis.set_title('Precision Vs Recall')
    
    plt.close()
    
    return axis if ax else fig

In [ ]:
def feature_importance_plot(importances, feature_labels, ax=None):
    fig, axis = (None, ax) if ax else plt.subplots(nrows=1, ncols=1, figsize=(5, 10))
    sns.barplot(x=importances, y=feature_labels, ax=axis)
    axis.set_title('Feature Importance Measures')
    
    plt.close()
    
    return axis if ax else fig

In [ ]:
def train_clf(clf, x_train, y_train, sample_weight=None, refit=False):
    train_time = 0
    
    try:
        if refit:
            raise NotFittedError
        y_pred_train = clf.predict(x_train)
    except NotFittedError:
        start = timeit.default_timer()
        
        if sample_weight is not None:
            clf.fit(x_train, y_train, sample_weight=sample_weight)
        else:
            clf.fit(x_train, y_train)
        
        end = timeit.default_timer()
        train_time = end - start
        
        y_pred_train = clf.predict(x_train)
    
    train_acc = accuracy_score(y_train, y_pred_train)
    return clf, y_pred_train, train_acc, train_time

In [ ]:
def model_memory_size(clf):
    return sys.getsizeof(pickle.dumps(clf))

In [ ]:
def report(clf, x_train, y_train, x_test, y_test, sample_weight=None,
           refit=False, importance_plot=False, confusion_labels=None,
           feature_labels=None, verbose=True):
    """ Trains the passed classifier if not already trained and reports
        various metrics of the trained classifier """
    
    dump = dict()
    
    ## Train if not already trained
    clf, train_predictions, \
    train_acc, train_time = train_clf(clf, x_train, y_train,
                                                     sample_weight=sample_weight,
                                                     refit=refit)
    ## Testing
    start = timeit.default_timer()
    test_predictions = clf.predict(x_test)
    end = timeit.default_timer()
    test_time = end - start
    
    test_acc = accuracy_score(y_test, test_predictions)
    y_probs = clf.predict_proba(x_test)[:, 1]
    
    roc_auc = roc_auc_score(y_test, y_probs)
    
    
    ## Model Memory
    model_mem = round(model_memory_size(clf) / 1024, 2)
    
    print(clf)
    print("\n=============================> TRAIN-TEST DETAILS <======================================")
    
    ## Metrics
    print(f"Train Size: {x_train.shape[0]} samples")
    print(f" Test Size: {x_test.shape[0]} samples")
    print("------------------------------------------")
    print(f"Training Time: {round(train_time, 3)} seconds")
    print(f" Testing Time: {round(test_time, 3)} seconds")
    print("------------------------------------------")
    print("Train Accuracy: ", train_acc)
    print(" Test Accuracy: ", test_acc)
    print("------------------------------------------")
    print(" Area Under ROC: ", roc_auc)
    print("------------------------------------------")
    print(f"Model Memory Size: {model_mem} kB")
    print("\n=============================> CLASSIFICATION REPORT <===================================")
    
    ## Classification Report
    clf_rep = classification_report(y_test, test_predictions, output_dict=True)
    
    print(classification_report(y_test, test_predictions,
                                target_names=confusion_labels))
    
    
    if verbose:
        print("\n================================> CONFUSION MATRIX <=====================================")
    
        ## Confusion Matrix HeatMap
        display(confusion_plot(confusion_matrix(y_test, test_predictions),
                               labels=confusion_labels))
        print("\n=======================================> PLOTS <=========================================")


        ## Variable importance plot
        fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(14, 10))
        roc_axes = axes[0, 0]
        pr_axes = axes[0, 1]
        importances = None

        if importance_plot:
            if not feature_labels:
                raise RuntimeError("'feature_labels' argument not passed "
                                   "when 'importance_plot' is True")

            try:
                importances = pd.Series(clf.feature_importances_,
                                        index=feature_labels) \
                                .sort_values(ascending=False)
            except AttributeError:
                try:
                    importances = pd.Series(clf.coef_.ravel(),
                                            index=feature_labels) \
                                    .sort_values(ascending=False)
                except AttributeError:
                    pass

            if importances is not None:
                # Modifying grid
                grid_spec = axes[0, 0].get_gridspec()
                for ax in axes[:, 0]:
                    ax.remove()   # remove first column axes
                large_axs = fig.add_subplot(grid_spec[0:, 0])

                # Plot importance curve
                feature_importance_plot(importances=importances.values,
                                        feature_labels=importances.index,
                                        ax=large_axs)
                large_axs.axvline(x=0)

                # Axis for ROC and PR curve
                roc_axes = axes[0, 1]
                pr_axes = axes[1, 1]
            else:
                # remove second row axes
                for ax in axes[1, :]:
                    ax.remove()
        else:
            # remove second row axes
            for ax in axes[1, :]:
                ax.remove()


        ## ROC and Precision-Recall curves
        clf_name = clf.__class__.__name__
        roc_plot(y_test, y_probs, clf_name, ax=roc_axes)
        precision_recall_plot(y_test, y_probs, clf_name, ax=pr_axes)

        fig.subplots_adjust(wspace=5)
        fig.tight_layout()
        display(fig)
    
    ## Dump to report_dict
    dump = dict(clf=clf, train_acc=train_acc, train_time=train_time,
                train_predictions=train_predictions, test_acc=test_acc,
                test_time=test_time, test_predictions=test_predictions,
                test_probs=y_probs, report=clf_rep, roc_auc=roc_auc,
                model_memory=model_mem)
    
    return clf, dump

In [ ]:
def compare_models(y_test=None, clf_reports=[], labels=[]):
    """ Compare evaluation metrics for the True Positive class [1] of 
        binary classifiers passed in the argument and plot ROC and PR curves.
        
        Arguments:
        ---------
        y_test: to plot ROC and Precision-Recall curves
        
        Returns:
        -------
        compare_table: pandas DataFrame containing evaluated metrics
                  fig: `matplotlib` figure object with ROC and PR curves """

    
    ## Classifier Labels
    default_names = [rep['clf'].__class__.__name__ for rep in clf_reports]
    clf_names =  labels if len(labels) == len(clf_reports) else default_names
    
    
    ## Compare Table
    table = dict()
    index = ['Train Accuracy', 'Test Accuracy', 'Overfitting', 'ROC Area',
             'Precision', 'Recall', 'F1-score', 'Support']
    for i in range(len(clf_reports)):
        train_acc = round(clf_reports[i]['train_acc'], 3)
        test_acc = round(clf_reports[i]['test_acc'], 3)
        clf_probs = clf_reports[i]['test_probs']
        roc_auc = clf_reports[i]['roc_auc']
        
        # Get metrics of True Positive class from sklearn classification_report
        true_positive_metrics = list(clf_reports[i]['report']["1"].values())
        
        table[clf_names[i]] = [train_acc, test_acc,
                               test_acc < train_acc, roc_auc] + true_positive_metrics
    
    table = pd.DataFrame(data=table, index=index)
    
    
    ## Compare Plots
    fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(15, 5))
    
    # ROC and Precision-Recall
    for i in range(len(clf_reports)):
        clf_probs = clf_reports[i]['test_probs']
        roc_plot(y_test, clf_probs, label=clf_names[i],
                 compare=True, ax=axes[0])
        precision_recall_plot(y_test, clf_probs, label=clf_names[i],
                              compare=True, ax=axes[1])
    # Plot No-Info classifier
    axes[0].plot([0,1], [0,1], linestyle='--', color='green')
        
    fig.tight_layout()
    plt.close()
    
    return table.T, fig

<a id="naive-bayes"></a>
## 5.2. Naive Bayes
The fundamental assumption made by Naive Bayes regarding the data is ***class conditional independence of features***. Sklearn provides different variants of Naive Bayes depending on whether the features follow a categorical distribution (CategoricalNB), normal distribution (GaussianNB), bernoulli distribution (BernoulliNB), multinomial distribution (MultinomialNB).

Since majority of the features are categorical and follow a categorical distribution, we will use CategoricalNB. Continuous features will be discretized.

In [ ]:
from sklearn.naive_bayes import CategoricalNB, GaussianNB 
from sklearn.preprocessing import KBinsDiscretizer, OrdinalEncoder

confusion_lbs = ['Edible', 'Poisonous']

nb_trans = [('ordinal', OrdinalEncoder(dtype=np.int64),
             list(x_train_cat.columns))]
nb_col_trans = ColumnTransformer(nb_trans, remainder='passthrough')

## Applying Column Transformer
x_train_nb = nb_col_trans.fit_transform(x_train_cat)
x_test_nb = nb_col_trans.transform(x_test_cat)

nb_clf = CategoricalNB()

nb_clf, nb_report = report(nb_clf, x_train_nb, y_train,
                           x_test_nb, y_test, refit=True,
                           confusion_labels=confusion_lbs)

<a id="logistic-regression"></a>
## 5.3. Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegressionCV
from sklearn.model_selection import GridSearchCV

confusion_lbs = ['Edible', 'Poisonous']

logit_cv = LogisticRegressionCV(class_weight='balanced', cv=5,
                                penalty='l2', random_state=0,
                                n_jobs=-1, refit=True,
                                scoring='f1', solver='liblinear')

logit_cv, logit_report = report(logit_cv, x_train, y_train,
                                x_test, y_test,
                                importance_plot=True,
                                feature_labels=feature_names,
                                confusion_labels=confusion_lbs)

<a id="k-nearest-neighbors"></a>
## 5.4. K-Nearest Neighbors

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(metric='minkowski', n_neighbors=69,
                           p=1, weights='distance', n_jobs=-1)

knn, knn_report = report(knn, x_train, y_train,
                         x_test, y_test,
                         importance_plot=True,
                         feature_labels=feature_names,
                         confusion_labels=confusion_lbs)

<a id="decision-tree"></a>
## 5.5. Decision Tree

In [ ]:
from sklearn.tree import DecisionTreeClassifier

decision_tree = DecisionTreeClassifier(ccp_alpha=0.0, class_weight='balanced',
                                       criterion='entropy', max_depth=4,
                                       random_state=0)

decision_tree, decision_tree_report = report(decision_tree, x_train, y_train,
                                             x_test, y_test,
                                             importance_plot=True,
                                             feature_labels=feature_names,
                                             confusion_labels=confusion_lbs)

<a id="decision-trees-with-bagging"></a>
## 5.6. Decision Trees with Bagging

In [ ]:
from sklearn.ensemble import BaggingClassifier

bagging_dtree = DecisionTreeClassifier(ccp_alpha=0.0, class_weight='balanced',
                                       criterion='entropy', max_depth=5,
                                       random_state=0)

bagging_clf = BaggingClassifier(base_estimator=bagging_dtree,
                                max_samples=0.05, n_estimators=110, n_jobs=-1,
                                random_state=0)

bagging_clf, bagging_clf_report = report(bagging_clf, x_train, y_train,
                                         x_test, y_test,
                                         feature_labels=feature_names,
                                         confusion_labels=confusion_lbs)

<a id="random-forests"></a>
## 5.7. Random Forests

In [ ]:
from sklearn.ensemble import RandomForestClassifier

random_forest = RandomForestClassifier(class_weight='balanced', criterion='entropy',
                                       max_depth=8, max_samples=0.7,
                                       n_estimators=230,
                                       n_jobs=-1, random_state=0)

random_forest, random_forest_report = report(random_forest, x_train, y_train,
                                             x_test, y_test,
                                             importance_plot=True,
                                             feature_labels=feature_names,
                                             confusion_labels=confusion_lbs)

<a id="decision-trees-with-adaboost"></a>
## 5.8. Decision Trees with AdaBoost
The default base estimator for `AdaBoostClassifier` is `DecisionTreeClassifier(max_depth=1)`

In [ ]:
from sklearn.ensemble import AdaBoostClassifier

boosting_dtree = DecisionTreeClassifier(class_weight='balanced', criterion='entropy',
                                        max_depth=1, random_state=0)
adaboot = AdaBoostClassifier(base_estimator=boosting_dtree,
                             n_estimators=390, learning_rate=0.095,
                             random_state=0)

adaboot, adaboot_report = report(adaboot, x_train, y_train,
                                 x_test, y_test,
                                 importance_plot=True,
                                 feature_labels=feature_names,
                                 confusion_labels=confusion_lbs)

<a id="linear-svc"></a>
## 5.9. Linear SVC

In [ ]:
from sklearn.svm import SVC

linear_svc = SVC(kernel='linear', probability=True,
                 class_weight='balanced', random_state=0)

linear_svc, linear_svc_report = report(linear_svc, x_train, y_train,
                                       x_test, y_test,
                                       importance_plot=True,
                                       feature_labels=feature_names,
                                       confusion_labels=confusion_lbs)

<a id="svm-with-rbf-kernel"></a>
## 5.10. SVM with RBF kernel

In [ ]:
rbf_svc = SVC(kernel='rbf', probability=True, class_weight='balanced', random_state=0)

rbf_svc, rbf_svc_report = report(rbf_svc, x_train, y_train,
                                 x_test, y_test,
                                 importance_plot=True,
                                 feature_labels=feature_names,
                                 confusion_labels=confusion_lbs)

<a id="xgboost"></a>
## 5.11. XGBoost

In [ ]:
from xgboost import XGBClassifier
from sklearn.utils import class_weight

cls_weight = (y_train.shape[0] - np.sum(y_train)) / np.sum(y_train)

xgb_clf = XGBClassifier(learning_rate=0.01, random_state=0,
                        scale_pos_weight=cls_weight, n_jobs=-1)
xgb_clf.fit(x_train, y_train)

xgb_clf, xgb_report = report(xgb_clf, x_train, y_train,
                             x_test, y_test,
                             importance_plot=True,
                             feature_labels=feature_names,
                             confusion_labels=confusion_lbs)

<a id="catboost"></a>
## 5.12. CatBoost
Cat boost performs better without One-hot encoding because it performs an internal categorical encoding that is similar to Leave One Out Encoding (LOOE). So, we can give the dataframe as input to the catboost classifier.

In [ ]:
from catboost import CatBoostClassifier

cat_feature_lbs = list(x_train_cat.columns)

catboost_clf = CatBoostClassifier(cat_features=cat_feature_lbs,
                                  l2_leaf_reg=120, depth=6,
                                  auto_class_weights='Balanced',
                                  iterations=200, learning_rate=0.16,
                                  use_best_model=True,
                                  early_stopping_rounds=150,
                                  eval_metric='F1', random_state=0)

catboost_clf.fit(x_train_cat, y_train, 
                 eval_set=(x_train_cat, y_train),
                 verbose=False)

catboost_clf, catboost_report = report(catboost_clf, x_train_cat, y_train,
                                       x_test_cat, y_test,
                                       importance_plot=True,
                                       feature_labels=cat_feature_lbs,
                                       confusion_labels=confusion_lbs)

<a id="model-comparison"></a>
# 6. Model Comparison
Since input data format for Naive Bayes and CatBoost are different, we will add them to the comparison manually.

In [ ]:
report_list = [nb_report, logit_report, knn_report, decision_tree_report,               
               bagging_clf_report, random_forest_report, adaboot_report,
               xgb_report, linear_svc_report, rbf_svc_report, catboost_report]
clf_labels = [rep['clf'].__class__.__name__ for rep in report_list]
clf_labels[-3], clf_labels[-2] = 'Linear SVC', 'RBF SVC'

<a id="evaluation-metrics"></a>
## 6.1. Evaluation Metrics

In [ ]:
compare_table, compare_plot = compare_models(y_test, clf_reports=report_list, labels=clf_labels)

compare_table.sort_values(by=['Overfitting', 'F1-score'],
                          ascending=[True, False])

<a id="roc-and-pr-curves"></a>
## 6.2. ROC and PR Curves

In [ ]:
compare_plot

***Thank You!!***